In [2]:
library(tidyverse)

# Apply a function to each element of a list or atomic vector

The map functions transform their input by applying a function to each element of a list or atomic vector and returning an object of the same length as the input.

* `map()` always returns a list. See the `modify()` family for versions that return an object of the same type as the input.

* `map_lgl()`, `map_int()`, `map_dbl()` and `map_chr()` return an atomic vector of the indicated type (or die trying).

* `map_dfr()` and `map_dfc()` return a data frame created by row-binding and column-binding respectively. They require `dplyr` to be installed.

* The returned values of `.f` must be of length one for each element of `.x`. If `.f` uses an extractor function shortcut, .default can be specified to handle values that are absent or empty. See `as_mapper()` for more on .default.

* `walk()` calls `.f` for its side-effect and returns the input `.x`.

```r
map(.x, .f, ...)

map_lgl(.x, .f, ...)

map_chr(.x, .f, ...)

map_int(.x, .f, ...)

map_dbl(.x, .f, ...)

map_raw(.x, .f, ...)

map_dfr(.x, .f, ..., .id = NULL)

map_dfc(.x, .f, ...)

walk(.x, .f, ...)`
```

**Arguments**
`.id`	
Either a string or NULL. If a string, the output will contain a variable with that name, storing either the name (if `.x` is named) or the index (if `.x` is unnamed) of the input. If NULL, the default, no variable will be created.

Only applies to `_dfr` variant.

# Examples

In [4]:
players <- c('VN Pikachu', 'Meomeo888', 'Tank Cao')

In [5]:
# Extract 2 last characters from each string

players %>% map_chr(str_sub, start = -2, end = -1)

[1] "hu" "88" "ao"

In [19]:
# Use an anonymous function
# Check if each string contains 'VN'
players %>% map_lgl(function(value) value %>% str_detect('VN'))

[1]  TRUE FALSE FALSE

In [20]:
# You can also use a formula
# Get 2 last characters of each string
players %>% map_chr(~ str_sub(., -2, -1))

[1] "hu" "88" "ao"

In [17]:
'VN' %in% 'VN Pikachu'

[1] FALSE

In [21]:
# Using set_names() with character vectors is handy to keep track
# of the original inputs:
set_names(c("foo", "bar")) %>% map_chr(paste0, ":suffix")


foo          bar 
"foo:suffix" "bar:suffix"

<hr>

Working with list

In [23]:
favorite_desserts <- list(Sophia = "banana bread", Eliott = "pancakes", Karina = "chocolate cake")

favorite_desserts

$Sophia
[1] "banana bread"

$Eliott
[1] "pancakes"

$Karina
[1] "chocolate cake"

In [24]:
favorite_desserts %>% map_chr(~ str_c(., ':suffix'))

Sophia                  Eliott                  Karina 
  "banana bread:suffix"       "pancakes:suffix" "chocolate cake:suffix"

In [29]:
# Extractor function
# Extract by name or position
# .default specifies value for elements that are missing or NULL

l1 <- list(list(a = 1L), list(a = NULL, b = 2L), list(b = 3L))

l1 %>% map('a', .default = '????')

[[1]]
[1] 1

[[2]]
[1] "????"

[[3]]
[1] "????"

In [30]:
l1 %>% map('b', .default = NA)

[[1]]
[1] NA

[[2]]
[1] 2

[[3]]
[1] 3

In [31]:
# Supply multiple values to index deeply into a list
l2 <- list(
  list(num = 1:3,     letters[1:3]),
  list(num = 101:103, letters[4:6]),
  list()
)

l2

[[1]]
[[1]]$num
[1] 1 2 3

[[1]][[2]]
[1] "a" "b" "c"


[[2]]
[[2]]$num
[1] 101 102 103

[[2]][[2]]
[1] "d" "e" "f"


[[3]]
list()

In [33]:
l2 %>% map(c(2, 2))

[[1]]
[1] "b"

[[2]]
[1] "e"

[[3]]
NULL

In [35]:
# Use a list to build an extractor that mixes numeric indices and names,
# and .default to provide a default value if the element does not exist

l2 %>% map(list('num', 1), .default = 'unknown')

[[1]]
[1] 1

[[2]]
[1] 101

[[3]]
[1] "unknown"

<hr>

Working with dataframe

In [36]:
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


In [37]:
# calculate the sum of each column
mtcars %>% map_dbl(sum)

mpg      cyl     disp       hp     drat       wt     qsec       vs 
 642.900  198.000 7383.100 4694.000  115.090  102.952  571.160   14.000 
      am     gear     carb 
  13.000  118.000   90.000

In [42]:
# A more realistic example: split a data frame into pieces, fit a
# model to each piece, summarise and extract R^2
mtcars %>% split(.$cyl) %>% map(~ lm(mpg ~ wt, data = .)) %>% map(summary) %>% map_dbl("r.squared")



4         6         8 
0.5086326 0.4645102 0.4229655

In [56]:
# If each element of the output is a data frame, use
# map_dfr to row-bind them together:
a <- matrix(1:4, 2, 2)
b <- matrix(1:4, 2, 2)
c <- matrix(1:4, 2, 2)

list('A' = a, 'B' = b, 'C' = c) %>% map_dfr(as.data.frame)

# Adding id column
list('A' = a, 'B' = b, 'C' = c) %>% map_dfr(as.data.frame, .id = 'unique key')

#If do not use a named list, the id for each group will be an integer
list(a, b, c) %>% map_dfr(~ as.data.frame(.), .id = 'unique key')

V1,V2
1,3
2,4
1,3
2,4
1,3
2,4


unique key,V1,V2
A,1,3
A,2,4
B,1,3
B,2,4
C,1,3
C,2,4


unique key,V1,V2
1,1,3
1,2,4
2,1,3
2,2,4
3,1,3
3,2,4
